In [1]:
class Heap:
    def __init__(self, key=lambda x:x):
        self.data = []
        self.key  = key

    @staticmethod
    def _parent(idx):
        return (idx-1)//2
        
    @staticmethod
    def _left(idx):
        return idx*2+1

    @staticmethod
    def _right(idx):
        return idx*2+2
    
    def heapify(self, idx=0):
        l = Heap._left(idx)
        r = Heap._right(idx)
        maxidx = idx
        if l < len(self) and self.key(self.data[l]) > self.key(self.data[idx]):
            maxidx = l
        if r < len(self) and self.key(self.data[r]) > self.key(self.data[maxidx]):
            maxidx = r
        if maxidx != idx:
            self.data[idx], self.data[maxidx] = self.data[maxidx], self.data[idx]
            self.heapify(maxidx)
            
    def add(self, x):
        self.data.append(x)
        idx = len(self.data)-1
        par = Heap._parent(idx)
        while idx > 0 and self.key(self.data[par]) < self.key(self.data[idx]):
            self.data[par], self.data[idx] = self.data[idx], self.data[par]
            idx, par = par, Heap._parent(par)
        
    def peek(self):
        return self.data[0]

    def pop(self):
        ret = self.data[0]
        self.data[0] = self.data[len(self.data)-1]
        del self.data[len(self.data)-1]
        self.heapify()
        return ret
    
    def __bool__(self):
        return len(self.data) > 0

    def __len__(self):
        return len(self.data)

    def __repr__(self):
        return repr(self.data)

In [2]:
class LinkedList:
    class Node:
        def __init__(self, val, prior=None, next=None):
            self.val = val
            self.prior = prior
            self.next  = next
    
    def __init__(self):
        self.head = LinkedList.Node(None) # sentinel node (never to be removed)
        self.head.prior = self.head.next = self.head # set up "circular" topology
        self.length = 0
        
        
    ### prepend and append, below, from class discussion
        
    def prepend(self, value):
        n = LinkedList.Node(value, prior=self.head, next=self.head.next)
        self.head.next.prior = self.head.next = n
        self.length += 1
        
    def append(self, value):
        n = LinkedList.Node(value, prior=self.head.prior, next=self.head)
        n.prior.next = n.next.prior = n
        self.length += 1
            
            
    ### subscript-based access ###
    
    def _normalize_idx(self, idx):
        nidx = idx
        if nidx < 0:
            nidx += len(self)
            if nidx < 0:
                nidx = 0
        return nidx
    
    def __getitem__(self, idx):
        """Implements `x = self[idx]`"""
        assert(isinstance(idx, int))
        nidx = self._normalize_idx(idx)
        if nidx >= self.length:
            raise IndexError
        n = self.head.next
        for _ in range(nidx):
            n = n.next
        return n.val

    def __setitem__(self, idx, value):
        """Implements `self[idx] = x`"""
        assert(isinstance(idx, int))
        nidx = self._normalize_idx(idx)
        if nidx >= self.length:
            raise IndexError
        n = self.head.next
        for _ in range(nidx):
            n = n.next
        n.val = value

    def __delitem__(self, idx):
        """Implements `del self[idx]`"""
        assert(isinstance(idx, int))
        nidx = self._normalize_idx(idx)
        if nidx >= self.length:
            raise IndexError
        n = self.head.next
        for _ in range(nidx):
            n = n.next
        n.prior.next, n.next.prior = n.next, n.prior
        self.length -= 1
        

    ### stringification ###
    
    def __str__(self):
        """Implements `str(self)`. Returns '[]' if the list is empty, else
        returns `str(x)` for all values `x` in this list, separated by commas
        and enclosed by square brackets. E.g., for a list containing values
        1, 2 and 3, returns '[1, 2, 3]'."""
        return '[' + ', '.join(str(x) for x in self) + ']'
        
    def __repr__(self):
        """Supports REPL inspection. (Same behavior as `str`.)"""
        return str(self)


    ### single-element manipulation ###
        
    def insert(self, idx, value):
        """Inserts value at position idx, shifting the original elements down the
        list, as needed. Note that inserting a value at len(self) --- equivalent
        to appending the value --- is permitted. Raises IndexError if idx is invalid."""
        nidx = self._normalize_idx(idx)
        if nidx > self.length:
            raise IndexError
        n = self.head.next
        for _ in range(nidx):
            n = n.next
        ins = LinkedList.Node(value, prior=n.prior, next=n)
        n.prior.next = n.prior = ins
        self.length += 1
    
    def pop(self, idx=-1):
        """Deletes and returns the element at idx (which is the last element,
        by default)."""
        nidx = self._normalize_idx(idx)
        if nidx >= self.length:
            raise IndexError
        n = self.head.next
        for _ in range(nidx):
            n = n.next
        n.prior.next, n.next.prior = n.next, n.prior
        self.length -= 1
        return n.val
    
    def remove(self, value):
        """Removes the first (closest to the front) instance of value from the
        list. Raises a ValueError if value is not found in the list."""
        n = self.head.next
        while n.val != value:
            n = n.next
            if n == self.head:
                raise ValueError
        n.prior.next, n.next.prior = n.next, n.prior
        self.length -= 1
    

    ### predicates (T/F queries) ###
    
    def __eq__(self, other):
        """Returns True if this LinkedList contains the same elements (in order) as
        other. If other is not an LinkedList, returns False."""
        # Try making it better later
        if len(self) != len(other) or not isinstance(other, LinkedList):
            return False
        for i in range(len(self)):
            if self[i] != other[i]:
                return False
        else:
            return True

    def __contains__(self, value):
        """Implements `val in self`. Returns true if value is found in this list."""
        for x in self:
            if x == value:
                return True
        else:
            return False


    ### queries ###
    
    def __len__(self):
        """Implements `len(self)`"""
        return self.length
    
    def min(self):
        """Returns the minimum value in this list."""
        ret = self[0]
        for x in self:
            if ret > x:
                ret = x
        return ret
    
    def max(self):
        """Returns the maximum value in this list."""
        ret = self[0]
        for x in self:
            if ret < x:
                ret = x
        return ret
    
    def index(self, value, i=0, j=None):
        """Returns the index of the first instance of value encountered in
        this list between index i (inclusive) and j (exclusive). If j is not
        specified, search through the end of the list for value. If value
        is not in the list, raise a ValueError."""
        high = len(self)
        idx = 0
        if j:
            high = self._normalize_idx(j)
        for x in self:
            if idx in range(i,high) and x==value:
                return idx
            idx += 1
        else:
            raise ValueError
    
    def count(self, value):
        """Returns the number of times value appears in this list."""
        cnt = 0
        for x in self:
            if x == value:
                cnt += 1
        return cnt

    
    ### bulk operations ###

    def __add__(self, other):
        """Implements `self + other_list`. Returns a new LinkedList
        instance that contains the values in this list followed by those 
        of other."""
        assert(isinstance(other, LinkedList))
        ret = LinkedList()
        ret.extend(self)
        ret.extend(other)
        return ret
    
    def clear(self):
        """Removes all elements from this list."""
        self.head.prior = self.head.next = self.head
        self.length = 0
        
    def copy(self):
        """Returns a new LinkedList instance (with separate Nodes), that
        contains the same values as this list."""
        ret = LinkedList()
        ret.extend(self)
        return ret

    def extend(self, other):
        """Adds all elements, in order, from other --- an Iterable --- to this list."""
        for x in other:
            self.append(x)

            
    ### iteration ###

    def __iter__(self):
        """Supports iteration (via `iter(self)`)"""
        n = self.head.next
        while n is not self.head:
            yield n.val
            n = n.next

In [138]:
import sys

class PathGraph:
    class Vertex:
        def __init__(self, v):
            self.key = v
            self.d = sys.maxsize

    def __init__(self, s, w, a):
        self.size = s
        self.weight = w
        self.adj = a
        self.pi = [None for vertex in range(s)]
        
    @staticmethod   
    def printSinglePareSP(path, s, f):
        print('shortest path from vertex {0} to {1}:\n'.format(s,f))
        for v in path:
            print (v.key)
            if v.key is f:
                break
            print (' to ')
        print ('\ncost: {}'.format(path.pop().d))
        
    def relax(self, u, v):
        print ('relaxing {1} with {0}'.format(v.key,u.key))
        print ('from {0} to {1}'.)
        if v.d > u.d + self.weight[u.key][v.key]:
            v.d = u.d + self.weight[u.key][v.key]
            self.pi[v.key] = u
            
    def dijkstraSinglePare(self, s, f):
        start = PathGraph.Vertex(s)
        start.d = 0
        find = PathGraph.Vertex(f)
        Q = Heap(lambda vert:-vert.d)
        Q.add(start)
        for key in range(1, self.size):
            vertex = PathGraph.Vertex(key)
            if key == f:
                Q.add(find)
                continue
            Q.add(vertex)
        # loop for solution
        while Q:
            u = Q.pop()
            print ('{0} is popped. Minimum is now {1}'.format(u.key,Q.peek().key))
            if u.d > find.d:
                break
            print (repr(u.key)+', '+repr(u.d))
            for v in self.adj[u.key]:
                print (repr(v.key)+', '+repr(v.d)+', ')
                self.relax(u, v)
                print (repr(v.key)+', '+repr(v.d)+', ')
        print ('to {0} from {1}:\n{1} '.format(s, f))
        path = self.pi[f]
        while path is not None:
            print ('-> ' + path.key)
        print ('cost: ')
#         path = []
#         temp = find
#         while temp is not None:
#             path.append(temp)
#             temp = self.pi[temp.key]
#         path.reverse()
#         PathGraph.printSinglePareSP(path, s, f)

In [139]:
size = 5
adjList = [LinkedList() for vertex in range(size)]
adjList[0].append(PathGraph.Vertex(1))
adjList[0].append(PathGraph.Vertex(3))
adjList[1].append(PathGraph.Vertex(2))
adjList[1].append(PathGraph.Vertex(3))
adjList[2].append(PathGraph.Vertex(4))
adjList[3].append(PathGraph.Vertex(1))
adjList[3].append(PathGraph.Vertex(2))
adjList[3].append(PathGraph.Vertex(4))
adjList[4].append(PathGraph.Vertex(0))
adjList[4].append(PathGraph.Vertex(2))
weight = [[0, 10, 0, 5, 0],
          [0,  0, 1, 2, 0],
          [0,  0, 0, 0, 4],
          [0,  3, 9, 0, 2],
          [7,  0, 6, 0, 0]]


In [140]:
G = PathGraph(size, weight, adjList)
G.dijkstraSinglePare(0,2)

0 is popped. Minimum is now 4
0, 0
1, 9223372036854775807, 
relaxing 0 with 1
1, 10, 
3, 9223372036854775807, 
relaxing 0 with 3
3, 5, 
4 is popped. Minimum is now 3
4, 9223372036854775807
0, 9223372036854775807, 
relaxing 4 with 0
0, 9223372036854775807, 
2, 9223372036854775807, 
relaxing 4 with 2
2, 9223372036854775807, 
3 is popped. Minimum is now 2
3, 9223372036854775807
1, 9223372036854775807, 
relaxing 3 with 1
1, 9223372036854775807, 
2, 9223372036854775807, 
relaxing 3 with 2
2, 9223372036854775807, 
4, 9223372036854775807, 
relaxing 3 with 4
4, 9223372036854775807, 
2 is popped. Minimum is now 1
2, 9223372036854775807
4, 9223372036854775807, 
relaxing 2 with 4
4, 9223372036854775807, 


IndexError: list index out of range

In [76]:
list(range(1,5))

[1, 2, 3, 4]